In [1]:
import gym

In [2]:
env = gym.make("CartPole-v1")
obs = env.reset()

In [3]:
obs

array([-0.00112524, -0.00323597,  0.04964419,  0.02379261])

In [4]:
env.render()

True

In [5]:
img = env.render(mode="rgb_array")
img.shape

(400, 600, 3)

In [6]:
env.action_space

Discrete(2)

In [7]:
action=1

In [8]:
obs, reward, done, info = env.step(action)

In [9]:
obs

array([-0.00118996,  0.19114019,  0.05012004, -0.25282301])

In [10]:
reward

1.0

In [11]:
done

False

In [12]:
info

{}

In [13]:
env.reset()

array([-0.03599531, -0.03283661, -0.01184384,  0.0384342 ])

In [14]:
env.render()

True

In [15]:
env.close()

In [16]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle <0 else 1

In [17]:
totals = []
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()

    for step in range(200):
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

In [18]:
import numpy as np
np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

(41.756, 8.785468911788374, 24.0, 67.0)

In [19]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [20]:
env.seed(42)
frames = []
obs = env.reset()

In [21]:
for step in range(200):
    img = env.render(mode="rgb_array")
    frames.append(img)
    action = basic_policy(obs)

    obs, reward, done, info = env.step(action)
    if done:
        break

In [22]:
def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch, 

In [23]:
def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = animation.FuncAnimation(fig, update_scene, fargs=(frames, patch), frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

In [24]:
plot_animation(frames)
plt.show()

In [25]:
env.close()

In [26]:
#  신경망 정책

import tensorflow as tf
import numpy as np

from tensorflow import keras


In [27]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


In [28]:
n_inputs = 4 # == env.observation_space.shape[0]

model = keras.models.Sequential([
    keras.layers.Dense(5, activation='elu', input_shape=[n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [29]:
def render_policy_net(model, n_max_steps = 200, seed = 42):
    framse = []

    env = gym.make("CartPole-v1")
    env.seed(seed)
    np.random.seed(seed)

    obs = env.reset()

    for step in range(n_max_steps):
        frames.append(env.render(mode="rgb_array"))
        left_proba = model.predict(obs.reshape(1, -1))
        action = int(np.random.rand() > left_proba)
        obs, reward, done, info = env.step(action)
        if done:
            break
    env.close()

    return frames

In [30]:
frames = render_policy_net(model)
plot_animation(frames)

In [31]:
n_environments = 50
n_iterations = 5000

envs = [gym.make("CartPole-v1") for _ in range(n_environments)]

for index, env in enumerate(envs):
    env.seed(index)

np.random.seed(42)

observations = [env.reset() for env in envs]
optimizer = keras.optimizers.RMSprop()
loss_fn = keras.losses.binary_crossentropy

for iteraion in range(n_iterations):
    # if angle < 0 we want proba(left) = 1., or else proba(left) = 0
    target_probas = np.array([([1.] if obs[2] < 0 else [0.]) for obs in observations])

    with  tf.GradientTape() as tape:
        left_probas = model(np.array(observations))
        loss = tf.reduce_mean(loss_fn(target_probas, left_probas))

    print("\rIteration: {}, Loss: {:.3f}".format(iteraion, loss.numpy()), end="")
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    actions = (np.random.rand(n_environments, 1) > left_probas.numpy()).astype(np.int32)

    for env_index, env in enumerate(envs):
        obs, reward, done, info = env.step(actions[env_index][0])
        observations[env_index] = obs if not done else env.reset()

for env in envs:
    env.close()

Iteration: 4999, Loss: 0.094

In [32]:
frames = render_policy_net(model)
plot_animation(frames)

### Policy Gradients

In [33]:
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        left_proba = model(obs[np.newaxis])
        action = (tf.random.uniform([1, 1]) > left_proba)
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba))
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(int(action[0, 0].numpy()))
    
    return obs, reward, done, info

In [34]:
def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_reward = []
    all_grads = []
    for episode in range(n_episodes):
        current_reward = []
        current_grads = []
        obs = env.reset()
        
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
            current_reward.append(reward)
            current_grads.append(grads)
            
            if done:
                break
                
        all_reward.append(current_reward)
        all_grads.append(current_grads)
    return all_reward, all_grads

In [35]:
def discount_rewards(rewards, dicount_factor):
    discounted = np.array(rewards)
    
    for step in range(len(rewards)-2, -1, -1):
        discounted[step] += discounted[step + 1] * dicount_factor
    return discounted

In [36]:
def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(rewards, discount_factor) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    
    return [(discount_rewards - reward_mean) / reward_std for discount_rewards in all_discounted_rewards]

In [37]:
discount_rewards([10, 0, -50], 0.8)

array([-22, -40, -50])

In [38]:
discount_and_normalize_rewards([[10,0,-50],[10,20]],0.8)

[array([-0.28435071, -0.86597718, -1.18910299]),
 array([1.26665318, 1.0727777 ])]

In [39]:
n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_factor = 0.95

In [40]:
optimizer = keras.optimizers.Adam(0.01)
loss_fn = keras.losses.binary_crossentropy

In [41]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(5, activation='elu', input_shape=[4]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [42]:
env = gym.make("CartPole-v1")
env.seed(42)

for iteraion in range(n_iterations):
    all_rewards, all_grads = play_multiple_episodes(env, n_episodes_per_update, n_max_steps, model, loss_fn)
    all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_factor)
    
    all_mean_grads = []
    
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index] 
            for episode_index, final_rewards in enumerate(all_final_rewards) 
            for step, final_reward in enumerate(final_rewards)], axis = 0)
        all_mean_grads.append(mean_grads)
        
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))
         
env.close()

KeyError: 0

In [ ]:
# env = gym.make("CartPole-v1")
# env.seed(42);

# for iteration in range(n_iterations):
#     all_rewards, all_grads = play_multiple_episodes(
#         env, n_episodes_per_update, n_max_steps, model, loss_fn)
#     total_rewards = sum(map(sum, all_rewards))                     # Not shown in the book
#     print("\rIteration: {}, mean rewards: {:.1f}".format(          # Not shown
#         iteration, total_rewards / n_episodes_per_update), end="") # Not shown
#     all_final_rewards = discount_and_normalize_rewards(all_rewards,
#                                                        discount_factor)
#     all_mean_grads = []
#     for var_index in range(len(model.trainable_variables)):
#         mean_grads = tf.reduce_mean(
#             [final_reward * all_grads[episode_index][step][var_index]
#              for episode_index, final_rewards in enumerate(all_final_rewards)
#                  for step, final_reward in enumerate(final_rewards)], axis=0)
#         all_mean_grads.append(mean_grads)
#     optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))

# env.close()

In [ ]:
#Markov Chains

In [43]:
np.random.seed(42)

transition_probabilities = [ # shape=[s, s']
        [0.7, 0.2, 0.0, 0.1],  # from s0 to s0, s1, s2, s3
        [0.0, 0.0, 0.9, 0.1],  # from s1 to ...
        [0.0, 1.0, 0.0, 0.0],  # from s2 to ...
        [0.0, 0.0, 0.0, 1.0]]  # from s3 to ...

n_max_steps = 50

In [44]:
def print_sequence():
    current_state = 0
    print("State:", end=" ")
    for step in range(n_max_steps):
        print(current_state, end=" ")
        if current_state == 3:
            break
        current_state = np.random.choice(range(4), p=transition_probabilities[current_state])
    else:
        print("...", end="")
    print()

In [45]:
for _ in range(10):
    print_sequence()

State: 0 0 3 
State: 0 1 2 1 2 1 2 1 2 1 3 
State: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
State: 0 3 
State: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
State: 0 1 3 
State: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 ...
State: 0 0 3 
State: 0 0 0 1 2 1 2 1 3 
State: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 


In [46]:
transition_probabilities = [ # shape=[s, a, s']
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
        [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
        [None, [0.8, 0.1, 0.1], None]]
rewards = [ # shape=[s, a, s']
        [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
        [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
        [[0, 0, 0], [+40, 0, 0], [0, 0, 0]]]
possible_actions = [[0, 1, 2], [0, 2], [1]]

In [48]:
Q_values = np.full((3,3), -np.inf)
for state, actions in enumerate(possible_actions):
    Q_values[state, actions] = 0.0

In [49]:
Q_values

array([[  0.,   0.,   0.],
       [  0., -inf,   0.],
       [-inf,   0., -inf]])